<a href="https://colab.research.google.com/github/AniruddhSrivatsa/AniruddhSrivatsa/blob/master/sms_spam_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("spamm.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df["Message"][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [5]:
df.shape

(5572, 2)

In [6]:
df.isna().sum()

Category    0
Message     0
dtype: int64

In [7]:
df["Message"]=df["Message"].str.lower()

In [8]:
df["Message"]=df["Message"].apply(lambda x: " ".join([info for info in x.split() if info.isalpha() ]))

In [9]:
df["Message"][0]

'go until jurong available only in bugis n great world la e cine there got amore'

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from nltk.corpus import stopwords

In [12]:
df["Message"]=df["Message"].apply(lambda x: " ".join([info for info in x.split() if info not in stopwords.words("english") ]))

In [13]:
df["Message"][0]

'go jurong available bugis n great world la e cine got amore'

removing single letter characters

In [14]:
df["Message"]=df["Message"].apply(lambda x: " ".join([info for info in x.split() if (len(info)!=1) ]))

In [ ]:
df["Message"][1]

'ok joking wif'

In [15]:
df.head()

,Category,Message
0,ham,go jurong available bugis great world la cine ...
1,ham,ok joking wif
2,spam,free entry wkly comp win fa cup final tkts may...
3,ham,dun say early already
4,ham,nah think goes lives around though


In [16]:
df["Category"]=df["Category"].map({"ham":1,"spam":0})

In [17]:
df1=pd.get_dummies(df["Category"])

In [18]:
df.head(10)

,Category,Message
0,1,go jurong available bugis great world la cine ...
1,1,ok joking wif
2,0,free entry wkly comp win fa cup final tkts may...
3,1,dun say early already
4,1,nah think goes lives around though
5,0,freemsg hey darling word like fun tb xxx std c...
6,1,even brother like speak treat like aids
7,1,per request melle minnaminunginte nurungu set ...
8,0,valued network customer selected receivea priz...
9,0,mobile months entitled update latest colour mo...


In [19]:
df=pd.concat([df,df1],axis=1)

In [21]:
from sklearn.model_selection import train_test_split

I am taking two models, one for general categorical classification using softmax and other for binary classification using sigmoid

In [23]:
X_train,X_test,y_train,y_test=train_test_split(df["Message"],df[[0,1]],stratify=df[[0,1]],test_size=0.2,random_state=0)

In [24]:
X1_train,X1_test,y1_train,y1_test=train_test_split(df["Message"],df["Category"],stratify=df["Category"],test_size=0.2,random_state=0)

In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [26]:
ar=list(X_train.astype("str"))
ar1=list(X_test.astype("str"))

In [27]:
ar3=list(X1_train.astype("str"))
ar4=list(X1_test.astype("str"))

In [29]:
tokens=Tokenizer(oov_token="<OOV>")
tokens.fit_on_texts(ar)
word_index=tokens.word_index

In [30]:
tokens1=Tokenizer(oov_token="<OOV>")
tokens1.fit_on_texts(ar3)
word_index1=tokens.word_index

In [32]:
len(word_index)

5357

In [33]:
len(word_index1)

5357

In [35]:
seq=tokens.texts_to_sequences(ar)
seq1=tokens.texts_to_sequences(ar1)

In [36]:
seq2=tokens.texts_to_sequences(ar3)
seq3=tokens.texts_to_sequences(ar4)

In [37]:
pad1=pad_sequences(seq,padding="post",maxlen=38,truncating="post")
pad2=pad_sequences(seq1,padding="post",maxlen=38,truncating="post")

In [38]:
pad3=pad_sequences(seq2,padding="post",maxlen=38,truncating="post")
pad4=pad_sequences(seq3,padding="post",maxlen=38,truncating="post")

In [39]:
pad1.shape

(4457, 38)

In [ ]:
mess=[len(i) for i in df["Message"]]

In [40]:
from tensorflow.keras.layers import Dense,Bidirectional,LSTM,Dropout,GlobalAveragePooling1D,Conv1D,Embedding,MaxPooling1D,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [41]:
callbacks=EarlyStopping(patience=3,monitor="val_accuracy")

In [63]:
model=Sequential()
model.add(Embedding(input_dim=len(word_index)+1,output_dim=180,input_length=38))
model.add(Conv1D(128,5,activation="relu"))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(150)))
model.add(Flatten())
model.add(Dense(2,activation="softmax"))

This is my first model

In [59]:
model1=Sequential()
model1.add(Embedding(input_dim=len(word_index1)+1,output_dim=180,input_length=38))
model1.add(Bidirectional(LSTM(180)))
model1.add(Dense(1,activation="sigmoid"))

This is my second model

In [64]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [60]:
model1.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [65]:
history=model.fit(pad1,y_train,epochs=10,callbacks=[callbacks],validation_split=0.1)

Epoch 1/10
126/126 [==============================] - 19s 151ms/step - loss: 0.1745 - accuracy: 0.9399 - val_loss: 0.0604 - val_accuracy: 0.9821
Epoch 2/10
126/126 [==============================] - 18s 143ms/step - loss: 0.0365 - accuracy: 0.9893 - val_loss: 0.0578 - val_accuracy: 0.9843
Epoch 3/10
126/126 [==============================] - 18s 144ms/step - loss: 0.0160 - accuracy: 0.9960 - val_loss: 0.0545 - val_accuracy: 0.9843
Epoch 4/10
126/126 [==============================] - 19s 149ms/step - loss: 0.0086 - accuracy: 0.9978 - val_loss: 0.0642 - val_accuracy: 0.9843
Epoch 5/10
126/126 [==============================] - 19s 154ms/step - loss: 0.0075 - accuracy: 0.9985 - val_loss: 0.0693 - val_accuracy: 0.9843


In [61]:
history1=model1.fit(pad3,y1_train,epochs=10,callbacks=[callbacks],validation_split=0.1)

Epoch 1/10
126/126 [==============================] - 27s 215ms/step - loss: 0.1930 - accuracy: 0.9359 - val_loss: 0.0798 - val_accuracy: 0.9821
Epoch 2/10
126/126 [==============================] - 27s 211ms/step - loss: 0.0524 - accuracy: 0.9843 - val_loss: 0.1041 - val_accuracy: 0.9753
Epoch 3/10
126/126 [==============================] - 27s 215ms/step - loss: 0.0307 - accuracy: 0.9923 - val_loss: 0.0822 - val_accuracy: 0.9731
Epoch 4/10
126/126 [==============================] - 27s 212ms/step - loss: 0.0145 - accuracy: 0.9965 - val_loss: 0.0871 - val_accuracy: 0.9776


In [66]:
model.evaluate(pad2,y_test)

35/35 [==============================] - 1s 28ms/step - loss: 0.1752 - accuracy: 0.9686


[0.17520971596240997, 0.9686098694801331]

In [62]:
model1.evaluate(pad4,y1_test)

35/35 [==============================] - 2s 54ms/step - loss: 0.0752 - accuracy: 0.9767


[0.07523591071367264, 0.9766815900802612]

The second model has higher test set accuracy

In [67]:
y_pred=model.predict(pad2)

In [68]:
y_pred1=model1.predict(pad4)

In [69]:
print(y_pred.shape,y_pred1.shape)

(1115, 2) (1115, 1)


In [70]:
y_pred1[0]

array([0.9914452], dtype=float32)

In [71]:
y_pred[0]

array([1.5526919e-05, 9.9998450e-01], dtype=float32)

In [72]:
predic=[]
for i in y_pred1:
  predic.append(i)

In [73]:
print(predic[:20],y1_test[:20])

[array([0.9914452], dtype=float32), array([0.99893105], dtype=float32), array([0.9997642], dtype=float32), array([0.99980736], dtype=float32), array([0.18629411], dtype=float32), array([0.9998697], dtype=float32), array([0.99997747], dtype=float32), array([0.0081709], dtype=float32), array([3.2627544e-05], dtype=float32), array([0.9998174], dtype=float32), array([0.9994658], dtype=float32), array([1.1020753e-05], dtype=float32), array([0.9998356], dtype=float32), array([0.9994371], dtype=float32), array([0.9999809], dtype=float32), array([7.099398e-05], dtype=float32), array([0.99989605], dtype=float32), array([4.876228e-05], dtype=float32), array([0.9998587], dtype=float32), array([0.91266274], dtype=float32)] 1589    1
5169    1
365     1
3821    1
714     1
3880    1
2178    1
684     0
4183    0
2982    1
204     1
2767    0
1679    1
299     1
5026    1
824     0
2813    1
1380    0
2177    1
2379    1
Name: Category, dtype: int64


Now enter your sentence

In [74]:
sent=["ok just joking bye see you tomorrow","free card worth rs 10000 won claim it now"]

In [75]:
see=tokens.texts_to_sequences(sent)
padded=pad_sequences(see,maxlen=38,truncating="post",padding="post")
opp=model1.predict(padded)
print(opp)
for i in opp:
  for j in i:
   print(j)
   if j>=0.50:
    print("ham message")
   else:
    print("spam message")



[[9.9962342e-01]
 [1.9824505e-04]]
0.9996234
ham message
0.00019824505
spam message
